<a href="https://colab.research.google.com/github/asakata0821/Aisa_INFO5731_-Fall2023/blob/main/Sakata_Exercise_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The third In-class-exercise (due on 11:59 PM 10/08/2023, 40 points in total)

The purpose of this exercise is to understand text representation.

Question 1 (10 points): Describe an interesting text classification or text mining task and explain what kind of features might be useful for you to build the machine learning model. List your features and explain why these features might be helpful. You need to list at least five different types of features.

In [ ]:
# Your answer here (no code for this question, write down your answer as detail as possible for the above questions):

'''
Please write you answer here:
I chose the Summarization system to answer this question because if we can summarize documents, it can save us a lot of time.
Also, I'm working on Japanese-American Incarceration history research now. I  want to apply this system to the research as well for more efficient work.
Note: I wrote codes for just extracting features for this assignmen, outputs are not accurate.
      I assume these codes will be fixed to give the right outputs for the future work.


There are 5 features which can be helpful for making the system better.
-TF-IDF
-Named Entity Recognition (NER) Tags
-Sentence Position
-Sentence Length
-Readability Scores

Reasons why they can be helpful are below:
TF-IDF: Based on the frequency of words, it assigns scores indicating the importance of each word.
        This helps us list those words and summarize the document according to their scores.

NER Tags: It identifies named entities. These entities often have higher scores of importance in TF-IDF.

Sentence Length: Shorter sentences tend to be useful for summarization, as they often capture key information.

Keyword Density: Based on the TF-IDF scores, I define the top 10 words as keywords for the passage (document).
                 Then calculate the keyword density to analyze the most important or prominent terms in the text.

Readability Score: Structurally simple and less complex sentences are better for summarization, which may enhance the overall readability of the summary.


'''

Question 2 (10 points): Write python code to extract these features you discussed above. You can collect a few sample text data for the feature extraction.

In [139]:
#For Feature 1
def calculate_tfidf_scores(texts):
    tfidf = TfidfVectorizer()
    features = tfidf.fit_transform(texts)
    word = tfidf.get_feature_names_out()
    tfidf_scores = features.toarray()[0]

    return word, tfidf_scores

#For Feature 2
def ner_tags(texts):
    doc = nlp(texts)
    ner_tags = [ent.label_ for ent in doc.ents]
    return ', '.join(ner_tags)

#For Feature 3
def Sentence_Length(texts):
    sentences = []
    word_counts = []

    def count_words(sentence):
        words = sentence.split()
        return len(words)

    #splitting a whole texts into sentences and storing
    for idx, sentence in enumerate(texts, start=1):
        sentence_parts = sentence.split('.')

        for part in sentence_parts:
            # Counting words
            word_count = count_words(part.strip())
            sentences.append(part.strip())
            word_counts.append(word_count)

    return sentences, word_counts


#For Feature 4
def keyword(tfidf_scores, word):
    #extracting the top 10 words and calculate TDTF-IDF scores for each
    top10_indices = tfidf_scores.argsort()[-10:][::-1]
    top10_scores = tfidf_scores[top10_indices]
    top10_words = [word[idx] for idx in top10_indices]

    return top10_words, top10_scores

def calculate_keyword_density(texts, keywords):
    densities = []

    for text in texts:
        words = text.lower().split()

        # For the case when there are no words
        if not words:
            densities.append(0)
            continue

        total_words = len(words)
        keyword_counter = 0

        #count the keyword numbers in sentences
        for keyword in keywords:
            keyword_counter += sum(1 for word in words if word == keyword)

        # Calculate the average of keyword density(%)
        keyword_density = (keyword_counter / total_words) * 100
        densities.append(keyword_density)

    return densities

#For Feature 5
def Readability_Scores(avg_words_per_sentence, avg_syllables_per_word):
    return 206.835 - 1.015 * avg_words_per_sentence - 84.6 * avg_syllables_per_word

def count_syllables(word):
    # Count syllable
    syllables = re.findall(r'[aeiouyAEIOUY]+', word)
    # Exclude parts where has 'e' in the end of words
    syllables = [syl for syl in syllables if not syl.endswith('e')]
    return len(syllables)

def calculate_avg_syllables_per_word(text):
    words = text.split()
    total_syllables = sum(count_syllables(word) for word in words if word)
    total_words = len(words)
    if total_words == 0:
        return 0
    return total_syllables / total_words




In [140]:
import spacy
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

# Load spaCy model for named entity recognition
nlp = spacy.load('en_core_web_sm')

#Collecting information by web scraping
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

url = 'https://www.jamsj.org/manabu/aiko-ebihara'

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
elements = soup.find_all(class_='sqs-block html-block sqs-block-html', id='block-yui_3_17_2_1_1587148327627_5945')

texts = []

for element in elements:
    paragraphs = element.find_all('p')
    for p in paragraphs:
        text = p.get_text()
        texts.append(text)



#1 TF-IDF
word, tfidf_scores = calculate_tfidf_scores(texts)
df = pd.DataFrame({
        'Word': word,
        'Feature 1: TF-IDF Scores': tfidf_scores,
    })


#2 NER tag
df['Feature 2: NER Tags'] = df['Word'].apply(ner_tags)
print(df)


#3 sentence length
sentences, word_counts = Sentence_Length(texts)


#4 keyword density
#for focusing more frequent words, extactthe top 10 words and define them as keywords
top10_words, top10_scores = keyword(tfidf_scores, word)

sentences = df2['Sentence']
keywords = top10_words

density = calculate_keyword_density(sentences, keywords)


df2 = pd.DataFrame({
    'Sentence': sentences,
    'Feature 3: Sentence Length': word_counts,
    'Feature 4: Keyword Density':density
}, index=range(1, len(sentences) + 1))


print(df2)

#5 readablity scores
words_per_sentence = df2['Feature 3: Sentence Length']
sum_of_words = sum(words_per_sentence)

#Removing sentences which has no characters (blank lines)
valid_sentences = df2[df2['Feature 3: Sentence Length'] != 0]
num_of_sentences = len(valid_sentences)

avg_words_per_sentence = sum_of_words / num_of_sentences

avg_syllables = calculate_avg_syllables_per_word(text)

fres_score = Readability_Scores(avg_words_per_sentence, avg_syllables)

print('************************************************************')
print("Feature 5: Readability Score",fres_score)





    Word       Feature 1: TF-IDF Scores Feature 2: NER Tags
0    2009      0.000000                  DATE              
1    27        0.000000                  CARDINAL          
2    above     0.074953                                    
3    ad        0.000000                                    
4    after     0.000000                                    
..     ...          ...                 ..                 
204  writings  0.000000                                    
205  wrote     0.074953                                    
206  wwii      0.074953                  PERSON            
207  years     0.095684                  DATE              
208  young     0.059094                                    

[209 rows x 3 columns]
   Sentence                                                                                                                                                                                                                                                     

Question 3 (10 points): Use any of the feature selection methods mentioned in this paper "Deng, X., Li, Y., Weng, J., & Zhang, J. (2019). Feature selection for text classification: A review. Multimedia Tools & Applications, 78(3)."

Select the most important features you extracted above, rank the features based on their importance in the descending order.

In [141]:
#I chose Word Embedding. This helps us to define classifications or topic categories for documents.

def Word_Embeddings(texts):
    words = []
    embeddings = []
    for text in texts:
        doc = nlp(text)
        for i in doc:
            words.append(i.text)
            embeddings.append(i.vector)
    return words, np.array(embeddings)

words, embeddings = Word_Embeddings(texts)

df3 = pd.DataFrame(embeddings, columns=[f'Dimension {i+1}' for i in range(embeddings.shape[1])])
df3.insert(0, 'Word', words)

print(df3)



#Importance Ranking
#1. TF-IDF (Term Frequency-Inverse Document Frequency)
#2. Named Entity Recognition (NER) Tags
#3. Word Embedding
#4. Sentence Length
#5. Readability Scores
#6. Sentence Position

#I made this ranking as considering the usability for Oral History Digital Achiving.
#TF-IDF came first because it can identify unique and significant terms in documents.
#Named Entity Recognition (NER) Tags is useful for capturing named entities which are very crucial for especially historical data.
#Word Embedding is more abstract representation compared to the top 2 methods.
#Sentence Length, Readability Scores, and Sentence Position are also significant but slightly less informative for capturing content essence in oral historical data.

    Word      Dimension 1  Dimension 2  Dimension 3  Dimension 4  Dimension 5  Dimension 6  Dimension 7  Dimension 8  Dimension 9  ...  Dimension 87  Dimension 88  Dimension 89  Dimension 90  Dimension 91  Dimension 92  Dimension 93  Dimension 94  Dimension 95  Dimension 96
0    This     0.196376    -0.225602    -0.216389     1.606050     0.116756    -0.023740    -0.677747     0.338733    -0.814684     ...  1.175289      0.024106      0.718536     -0.875431     -0.846864     -0.211391      0.872998     -0.635098     -1.227015     -0.484034    
1    story   -0.626435     0.170139    -1.241805    -0.829305     0.762212    -0.108243     2.029763     1.126165    -0.431281     ...  0.072290      0.141888     -1.006470     -0.326446     -0.420018     -0.466750      1.227321      0.568568     -0.822245      1.637805    
2    of       1.597100    -0.232040     0.284133    -0.391191    -0.869625    -0.926237    -0.092524    -0.670386     0.764744     ... -1.318537      1.599347     -0.605079   

Question 4 (10 points): Write python code to rank the text based on text similarity. Based on the text data you used for question 2, design a query to match the most relevant docments. Please use the BERT model to represent both your query and the text data, then calculate the cosine similarity between the query and each text in your data. Rank the similary with descending order.

In [120]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.5 MB/s eta 0:00:00


In [142]:
#This is also I assume that I'll fix for the future work. For now, outputs are not accurate.


import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# From previous df, extracting only sentences which are usable
sentences = list(valid_sentences['Sentence'])
query = "place"

# Tokenize the query and sentences
tokenized_query = tokenizer(query, return_tensors='pt')
tokenized_sentences = tokenizer(sentences, return_tensors='pt', padding=True, truncation=True)

# Get embeddings using BERT model
with torch.no_grad():
    query_embeddings = model(**tokenized_query).last_hidden_state.mean(dim=1).squeeze()
    sentence_embeddings = model(**tokenized_sentences).last_hidden_state.mean(dim=1).squeeze()

# Reshape the embeddings to align for cosine similarity calculation
query_embeddings = query_embeddings.reshape(1, -1)
sentence_embeddings = sentence_embeddings.reshape(len(sentences), -1)

# Compute cosine similarity
similarity = cosine_similarity(query_embeddings, sentence_embeddings).squeeze()

ranking = np.argsort(similarity)[::-1]

for i, rank in enumerate(ranking):
    print(f"{i+1}: Similarity = {similarity[rank]:.4f}, Sentence: {sentences[rank]}")


1: Similarity = 0.3772, Sentence: Mr
2: Similarity = 0.2966, Sentence: For three years, the Williams raised Aiko in their home, with frequent visits and check-ins with the Tanakas
3: Similarity = 0.2938, Sentence: Williams even carried a photo of Aiko with him until the day he died
4: Similarity = 0.2869, Sentence: But when the war broke out, anti-Japanese sentiment quietly but swiftly permeated the town
5: Similarity = 0.2824, Sentence: With Aiko on the way, there was simply no room for a newborn baby and an arrangement was made for a family in town, the Williams, to care for her for a few years
6: Similarity = 0.2780, Sentence: This story of Aiko Ebihara really begins in the friendship forged between two families living in Salem, Oregon prior to the start of WWII
7: Similarity = 0.2777, Sentence: I love my wife, I love my children, I love my home and I love my United States
8: Similarity = 0.2718, Sentence: The arrangement worked out so well that Velora, the only child of Edward and 